## 1) Leitura dos CSVs

In [19]:
import pandas as pd
import numpy as np

In [20]:
order          = pd.read_csv("Order.csv", sep=";")
campaign       = pd.read_csv("Campaign.csv", sep=";")
campaign_queue = pd.read_csv("CampaignQueue.csv", sep=";")
customer       = pd.read_csv("Customer.csv", sep=";")
for nome, df in {"Order":order,"Campaign":campaign,"CampaignQueue":campaign_queue,"Customer":customer}.items():
    print(f"{nome}: {df.shape[0]} linhas x {df.shape[1]} colunas")

Order: 3500 linhas x 23 colunas
Campaign: 2000 linhas x 14 colunas
CampaignQueue: 5000 linhas x 16 colunas
Customer: 1000 linhas x 17 colunas


## 2) Conversões básicas de tipo (datas, números, textos)

### 2.1) Customer — datas e textos

In [21]:
customer_cl = customer.copy()

for col in ["dateOfBirth", "createdAt", "updatedAt", "enrichedAt"]:
    if col in customer_cl.columns:
        customer_cl[col] = pd.to_datetime(customer_cl[col], errors="coerce", dayfirst=True)

for col in ["name", "gender", "externalCode", "email", "phone"]:
    if col in customer_cl.columns:
        customer_cl[col] = customer_cl[col].astype(str).str.strip()

if "gender" in customer_cl.columns:
    customer_cl["gender"] = customer_cl["gender"].str.upper()

### 2.2) Campaign — datas e textos

In [22]:
campaign_cl = campaign.copy()

for col in ["createdAt", "updatedAt", "_createdAt"]:
    if col in campaign_cl.columns:
        campaign_cl[col] = pd.to_datetime(campaign_cl[col], errors="coerce", dayfirst=True)

for col in ["badge", "name", "description", "type", "storeId"]:
    if col in campaign_cl.columns:
        campaign_cl[col] = campaign_cl[col].astype(str).str.strip()

### 2.3) CampaignQueue — datas, textos e status numérico

In [23]:
campaign_queue_cl = campaign_queue.copy()

for col in ["scheduledAt", "sendAt", "createdAt", "updatedAt"]:
    if col in campaign_queue_cl.columns:
        campaign_queue_cl[col] = pd.to_datetime(campaign_queue_cl[col], errors="coerce", dayfirst=True)

for col in ["message", "response", "storeId", "storeInstanceId"]:
    if col in campaign_queue_cl.columns:
        campaign_queue_cl[col] = campaign_queue_cl[col].astype(str).str.strip()

if "status" in campaign_queue_cl.columns:
    campaign_queue_cl["status"] = pd.to_numeric(campaign_queue_cl["status"], errors="coerce")

### 2.4) Order — datas, números e textos

In [24]:
order_cl = order.copy()

for col in ["createdAt", "scheduledAt", "updatedAt", "_createdAt"]:
    if col in order_cl.columns:
        order_cl[col] = pd.to_datetime(order_cl[col], errors="coerce", dayfirst=True)

if "totalAmount" in order_cl.columns:
    order_cl["totalAmount_num"] = (
        order_cl["totalAmount"]
        .astype(str)
        .str.replace(".", "", regex=False)
        .str.replace(",", ".", regex=False)
    )
    order_cl["totalAmount_num"] = pd.to_numeric(order_cl["totalAmount_num"], errors="coerce")

for col in ["salesChannel", "engineName", "engineType", "orderType", "orderTiming", "version"]:
    if col in order_cl.columns:
        order_cl[col] = order_cl[col].astype(str).str.strip()

## 3) Tratamento de valores nulos

### 3.1) Order

In [25]:
for col in ["engineName", "engineType", "orderType", "orderTiming", "salesChannel", "version"]:
    if col in order_cl.columns:
        order_cl[col] = order_cl[col].fillna("Nao informado")

if "extraInfo" in order_cl.columns:
    order_cl["extraInfo"] = order_cl["extraInfo"].fillna("")

### 3.2) Campaign

In [26]:
for col in ["badge", "description", "name", "type"]:
    if col in campaign_cl.columns:
        campaign_cl[col] = campaign_cl[col].replace({"nan": np.nan}).fillna("Nao informado")

### 3.3) CampaignQueue

In [27]:
if "message" in campaign_queue_cl.columns:
    campaign_queue_cl["message"] = campaign_queue_cl["message"].fillna("Nao informado")

if "response" in campaign_queue_cl.columns:
    campaign_queue_cl["response"] = campaign_queue_cl["response"].replace({"": np.nan}).fillna("Nao informado")

### 3.4) Customer

In [28]:
if "externalCode" in customer_cl.columns:
    customer_cl["externalCode"] = customer_cl["externalCode"].replace({"": np.nan}).fillna("Nao informado")

## 4) Padronização e criação de novos campos

In [29]:
# Transforma storeId maiúsculo em todas as bases que possuem essa coluna
for df in (order_cl, campaign_cl, campaign_queue_cl):
    if "storeId" in df.columns:
        df["storeId"] = df["storeId"].astype(str).str.strip().str.upper()

In [30]:
# Campo textual para enriquecimento de clientes
if "isEnriched" in customer_cl.columns:
    customer_cl["isEnriched_text"] = customer_cl["isEnriched"].map(
        {True: "Enriquecido", False: "Nao Enriquecido"}
    )

In [31]:
# Status legíveis (sem acentos para evitar problemas de encoding)
map_status_customer = {1: "Ativo", 2: "Inativo"}
map_status_campaign = {1: "Rascunho", 2: "Agendada", 3: "Ativa", 4: "Concluida"}
map_status_queue = {
    1: "Pendente",
    2: "Enviada",
    3: "Entregue",
    4: "Lida",
    5: "Erro",
    6: "Cancelada",
}

if "status" in customer_cl.columns and customer_cl["status"].dtype != "O":
    customer_cl["status_desc"] = customer_cl["status"].map(map_status_customer).fillna("Nao informado")

if "status" in campaign_cl.columns and campaign_cl["status"].dtype != "O":
    campaign_cl["status_desc"] = campaign_cl["status"].map(map_status_campaign).fillna("Nao informado")

if "status" in campaign_queue_cl.columns and campaign_queue_cl["status"].dtype != "O":
    campaign_queue_cl["status_desc"] = campaign_queue_cl["status"].map(map_status_queue).fillna("Nao informado")

In [32]:
# Criação de campo de mês (_mes) em todas as bases

# Order: mês a partir de createdAt
if "createdAt" in order_cl.columns:
    order_cl["_mes"] = order_cl["createdAt"].dt.to_period("M").astype(str)

# Campaign: preencher createdAt nulo com updatedAt e criar _mes
if "createdAt" in campaign_cl.columns:
    if "updatedAt" in campaign_cl.columns:
        campaign_cl["createdAt_filled"] = campaign_cl["createdAt"].fillna(campaign_cl["updatedAt"])
    else:
        campaign_cl["createdAt_filled"] = campaign_cl["createdAt"]
    campaign_cl["_mes"] = pd.to_datetime(
        campaign_cl["createdAt_filled"], errors="coerce"
    ).dt.to_period("M").astype(str)

# CampaignQueue: mês considerando sendAt / scheduledAt / createdAt
if "sendAt" in campaign_queue_cl.columns:
    base_dt = campaign_queue_cl["sendAt"]
    if "scheduledAt" in campaign_queue_cl.columns:
        base_dt = base_dt.fillna(campaign_queue_cl["scheduledAt"])
    if "createdAt" in campaign_queue_cl.columns:
        base_dt = base_dt.fillna(campaign_queue_cl["createdAt"])
    campaign_queue_cl["_mes"] = pd.to_datetime(
        base_dt, errors="coerce"
    ).dt.to_period("M").astype(str)

## 5) Inferência de gênero (somente faltantes ou inválidos)

In [33]:
pip install gender-guesser

In [34]:
try:
    import gender_guesser.detector as gender
    det = gender.Detector(case_sensitive=False)
    pode_inferir = True
except Exception:
    print("gender_guesser não disponível. Prosseguindo sem inferência automática.")
    det = None
    pode_inferir = False

def pegar_primeiro_nome(nome):
    if pd.isna(nome):
        return None
    texto = str(nome).strip()
    if not texto:
        return None
    return texto.split()[0]

def traduzir_guess(g):
    if g in ["male", "mostly_male"]:
        return "M"
    if g in ["female", "mostly_female"]:
        return "F"
    return "Nao informado"

if "gender" in customer_cl.columns and "name" in customer_cl.columns:
    customer_cl["gender"] = customer_cl["gender"].str.upper()
    validos = customer_cl["gender"].isin(["M", "F", "O"])
    mascara_invalido = ~validos

    if pode_inferir and mascara_invalido.any():
        primeiros = customer_cl.loc[mascara_invalido, "name"].apply(pegar_primeiro_nome)
        palpites = primeiros.apply(
            lambda n: det.get_gender(n) if isinstance(n, str) else "unknown"
        )
        inferido = palpites.apply(traduzir_guess)
        customer_cl.loc[mascara_invalido, "gender_inferred"] = inferido
        customer_cl.loc[mascara_invalido, "gender_final"] = np.where(
            inferido.isin(["M", "F"]), inferido, "Nao informado"
        )
    else:
        customer_cl.loc[mascara_invalido, "gender_final"] = "Nao informado"

    customer_cl.loc[validos, "gender_final"] = customer_cl.loc[validos, "gender"]
    customer_cl.loc[customer_cl["gender"] == "O", "gender_final"] = "O"
    customer_cl["gender_clean"] = customer_cl["gender_final"]

    print("Distribuição (gender_clean):")
    print(customer_cl["gender_clean"].value_counts(dropna=False))

Distribuição (gender_clean):
gender_clean
F                344
M                309
O                263
Nao informado     84
Name: count, dtype: int64


## 6) Validações finais (nulos, duplicados, integridade)

In [46]:
def resumo_base(df, nome):
    print(f"\n>>> {nome}")
    print("Linhas/colunas:", df.shape)
    print("Duplicados:", df.duplicated().sum())
    print("Nulos por coluna:")
    print(df.isna().sum())

resumo_base(order_cl, "Order")
resumo_base(campaign_cl, "Campaign")
resumo_base(campaign_queue_cl, "CampaignQueue")
resumo_base(customer_cl, "Customer")

# Checagem de integridade

# Carrega a base de estabelecimentos
estab = pd.read_csv("Estabelecimentos.csv", sep=";")

# CampaignQueue -> Campaign
if set(["campaignId"]).issubset(campaign_queue_cl.columns) and "id" in campaign_cl.columns:
    faltando_camp = (~campaign_queue_cl["campaignId"].isin(campaign_cl["id"])).sum()
    print("\nCampaignQueue com campaignId inexistente:", faltando_camp)

# CampaignQueue -> Customer
if set(["customerId"]).issubset(campaign_queue_cl.columns) and "id" in customer_cl.columns:
    faltando_cli = (~campaign_queue_cl["customerId"].isin(customer_cl["id"])).sum()
    print("CampaignQueue com customerId inexistente:", faltando_cli)

# Order -> Customer
if "customer" in order_cl.columns and "id" in customer_cl.columns:
    faltando_cli_order = (~order_cl["customer"].isin(customer_cl["id"])).sum()
    print("\nOrder com customer inexistente:", faltando_cli_order)

# Order -> Estabelecimentos
if "companyId" in order_cl.columns and "establishment_id" in estab.columns:
    faltando_estab_order = (~order_cl["companyId"].isin(estab["establishment_id"])).sum()
    print("Order com companyId inexistente em estabelecimentos:", faltando_estab_order)

# Customer -> Estabelecimentos
if "companyId" in customer_cl.columns and "establishment_id" in estab.columns:
    faltando_estab_cust = (~customer_cl["companyId"].isin(estab["establishment_id"])).sum()
    print("\nCustomer com companyId inexistente em estabelecimentos:", faltando_estab_cust)

# Campaign -> Estabelecimentos
if "storeId" in campaign_cl.columns and "establishment_id" in estab.columns:
    faltando_estab_camp = (~campaign_cl["storeId"].isin(estab["establishment_id"])).sum()
    print("Campaign com storeId inexistente em estabelecimentos:", faltando_estab_camp)



>>> Order
Linhas/colunas: (3500, 25)
Duplicados: 0
Nulos por coluna:
id                         0
companyId                  0
containerId                0
createdAt                  0
customer                   0
displayId                  0
engineId                   0
engineName                 0
engineType                 0
extraInfo                  0
integrated                 0
integrationId              0
isTest                     0
orderTiming                0
orderType                  0
salesChannel               0
scheduledAt             1729
status                     0
preparationTime            0
takeOutTimeInSeconds       0
totalAmount                0
updatedAt                  0
version                    0
totalAmount_num            0
_mes                       0
dtype: int64

>>> Campaign
Linhas/colunas: (2000, 17)
Duplicados: 0
Nulos por coluna:
id                     0
segmentId              0
templateId             0
storeId                0
name               

## 7) Exportar dados tratados para novos CSVs

In [36]:
order_cl.to_csv("Order_clean.csv", index=False, sep=";")
campaign_cl.to_csv("Campaign_clean.csv", index=False, sep=";")
campaign_queue_cl.to_csv("CampaignQueue_clean.csv", index=False, sep=";")
customer_cl.to_csv("Customer_clean.csv", index=False, sep=";")

print("Arquivos tratados exportados!")

Arquivos tratados exportados!
